# pymovements in 10 minutes

## What you will learn in this tutorial:

* how to download one of the publicly available datasets
* how to load a subset of the data into your memory
* how to transform pixel coordinates into degrees of visual angle
* how to transform positional data into velocity data
* how to detect fixations by using the I-VT algorithm
* how to detect saccades by using the microsaccades algorithm
* how to compute additional event properties for your analysis
* how to save your preprocessed data
* how to plot the main saccadic sequence from your data

## Downloading one of the public datasets

We import `pymovements` as the alias `pm` for convenience.

In [ ]:
import polars as pl

import pymovements as pm

pymovements provides a library of publicly available datasets.

You can browse through the available dataset definitions here:
[Datasets](https://pymovements.readthedocs.io/en/latest/reference/pymovements.datasets.html#module-pymovements.datasets)

For this tutorial we will limit ourselves to the `ToyDataset` due to its minimal space requirements.

Other datasets can be downloaded by simply replacing `ToyDataset` with one of the other available datasets.

We can initialize and download by passing the desired dataset name as a string argument.

Additionally we need the root directory path of your data.

In [ ]:
dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')
dataset.download()

Our downloaded dataset will be placed in new a directory with the name of the dataset:

In [ ]:
dataset.path

Archive files are automatically extracted into the path specified by `Dataset.paths.raw`:

In [ ]:
dataset.paths.raw

## Loading in your data into memory

Next we load our dataset into memory to be able to work with it:

In [ ]:
dataset.load()

This way we fill two attributes with data.
First we have the `fileinfo` attribute which holds all the basic information for files:

In [ ]:
dataset.fileinfo['gaze'].head()

We notice that for each filepath a `text_id` and `page_id` is specified.

We have also loaded our gaze data into the dataframes in the `gaze` attribute:

In [ ]:
dataset.gaze[0]

Apart from some trial identifier columns we see the columns `time` and `pixel`.

The last two columns refer to the pixel coordinates at the timestep specified by `time`.


We are also able to just take a subset of the data by specifying values of the fileinfo columns.
The key refers to the column in the `fileinfo` dataframe.
The values in the dictionary can be of type `bool`, `int`,  `float` or `str`, but also lists and ranges 


In [ ]:
subset = {
    'text_id': 0,
    'page_id': [0, 1],
}
dataset.load(subset=subset)

dataset.fileinfo

Now we selected only a small subset of our data.

## Preprocessing raw gaze data

We now want to preprocess our gaze data by transforming pixel coordinates into degrees of visual angle and then computing velocity data from our positional data.

In [ ]:
dataset.pix2deg()

dataset.gaze[0]

We notice that a new column has appeared: `position`.
This column specifies the position coordinates in degrees of visual angle (dva).

For transforming our positional data into velocity data we will use the *Savitzky-Golay* differentiation filter.

We can also specify some additional parameters for this method:

In [ ]:
dataset.pos2vel(method='savitzky_golay', degree=2, window_length=7)

dataset.gaze[0]

There is also the more general apply() method, which can be used to apply both transformation and event detection methods.

In [ ]:
dataset.apply('pos2acc', degree=2, window_length=7)

dataset.gaze[0]

## Detecting events

Now let's detect some events.

First we will detect fixations using the I-VT algorithm using its default parameters:

In [ ]:
dataset.detect_events('ivt')

dataset.events[0]

Next we detect some saccades. This time we don't use the default parameters but specify our own:

In [ ]:
dataset.detect_events('microsaccades', minimum_duration=8)

dataset.events[0].frame.filter(pl.col('name') == 'saccade').head()

We can also use the more general interface of the apply() method:

In [ ]:
dataset.apply('idt', dispersion_threshold=2.7, name='fixation.ivt')

dataset.events[0].frame.filter(pl.col('name') == 'fixation.ivt').head()

## Computing event properties

The event dataframe currently only holds the `name`, `onset`, `offset` and `duration` of an event (additionally we have some more identifier columns at the beginning).

We now want to compute some additional properties for each event.
Event properties are things like peak velocity, amplitude and dispersion during an event.

We start out with computing the dispersion:

In [ ]:
dataset.compute_event_properties("dispersion")

dataset.events[0]

We notice that a new column with the name `dispersion` has appeared in the event dataframe.

We can also pass a list of properties to compute all of our desired properties in a single run.
Let's add the amplitude and peak velocity:

In [ ]:
dataset.compute_event_properties(["amplitude", "peak_velocity"])

dataset.events[0]

## Plotting our data

*pymovements* provides a range of plotting functions.

You can browse through the available plotting functions here:
[Plotting](https://pymovements.readthedocs.io/en/latest/reference/pymovements.plotting.html#module-pymovements.plotting)

In this this tutorial we will plot the saccadic main sequence of our data.

In [ ]:
pm.plotting.main_sequence_plot(dataset.events[0])

## Saving and loading your dataframes

If we want to save interim results we can simply use the `save()` method like this:

In [ ]:
dataset.save()

Let's test this out by initializing a new `PublicDataset` object in the same directory and loading in the preprocessed gaze and event data.

This time we don't need to download anything.

In [ ]:
preprocessed_dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')

dataset.load(events=True, preprocessed=True, subset=subset)

display(dataset.gaze[0])
display(dataset.events[0])